<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-422-salleh/Exercises/day-13/AI-Ethics/Production-Ready-Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Set: Operationalizing Ethics in AI Development & Deployment — Recruitment Screening Context

We’ll simulate an AI model that screens job applicants for shortlisting. Your tasks will focus on embedding and operationalizing ethics both during development and after deployment.

1. Dataset Creation (Run in Colab)

In [4]:
import pandas as pd
import numpy as np

np.random.seed(123)

# Synthetic Recruitment Dataset
n = 1200
genders = ["Male", "Female", "Other"]
ethnicities = ["Group_A", "Group_B", "Group_C"]
education_levels = ["High School", "Bachelors", "Masters", "PhD"]

data = pd.DataFrame({
    "Applicant_ID": range(1, n+1),
    "Age": np.random.randint(20, 60, n),
    "Gender": np.random.choice(genders, n, p=[0.5, 0.45, 0.05]),
    "Ethnicity": np.random.choice(ethnicities, n, p=[0.4, 0.35, 0.25]),
    "Education": np.random.choice(education_levels, n, p=[0.3, 0.4, 0.2, 0.1]),
    "Years_Experience": np.random.randint(0, 20, n),
    "Skill_Score": np.random.randint(40, 100, n)  # Out of 100
})

# Introduce a bias pattern:
# Applicants with Master's or PhD and Skill_Score > 70 more likely to be shortlisted
data["Shortlisted"] = np.where(
    (data["Education"].isin(["Masters", "PhD"])) & (data["Skill_Score"] > 70),
    np.random.choice([1, 0], n, p=[0.75, 0.25]),
    np.random.choice([1, 0], n, p=[0.35, 0.65])
)

# Extra: Slight advantage for Ethnicity = Group_A
mask_group_a = data["Ethnicity"] == "Group_A"
boost = np.random.choice([1, 0], size=n, p=[0.55, 0.45])
data["Shortlisted"] = np.where(mask_group_a, boost, data["Shortlisted"])

data.head()

,Applicant_ID,Age,Gender,Ethnicity,Education,Years_Experience,Skill_Score,Shortlisted
0,1,22,Male,Group_B,Bachelors,12,87,0
1,2,48,Other,Group_B,Bachelors,12,89,1
2,3,54,Male,Group_B,Bachelors,19,45,0
3,4,58,Male,Group_C,Masters,8,41,0
4,5,37,Male,Group_C,Bachelors,16,48,0


#Part A — Operationalizing Ethics in Development

#Task 1 — Bias Detection in Development

    Train a baseline model to predict Shortlisted.
    Measure positive prediction rates by Gender and Ethnicity.
    Identify potential bias sources.


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [7]:
# Prepare data
X = pd.get_dummies(data.drop(columns=["Applicant_ID","Shortlisted"]), drop_first=True)
y = data["Shortlisted"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train baseline model
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

Baseline Accuracy: 0.5361111111111111


In [12]:
# Bias check by Gender and Ethnicity
# Merge back with sensitive attributes for bias check
df_test = X_test.copy()
df_test["y_true"] = y_test
df_test["y_pred"] = y_pred
df_test["Gender"] = data.loc[y_test.index, "Gender"]
df_test["Ethnicity"] = data.loc[y_test.index, "Ethnicity"]

# Bias by Gender
print("\nPositive prediction rate by Gender:")
print(df_test.groupby("Gender")["y_pred"].mean())

# Bias by Ethnicity
print("\nPositive prediction rate by Ethnicity:")
print(df_test.groupby("Ethnicity")["y_pred"].mean())


Positive prediction rate by Gender:
Gender
Female    0.387500
Male      0.347826
Other     0.812500
Name: y_pred, dtype: float64

Positive prediction rate by Ethnicity:
Ethnicity
Group_A    0.522388
Group_B    0.261194
Group_C    0.369565
Name: y_pred, dtype: float64


#Task 2 — Fairness Testing in Development

Implement a pre‑deployment fairness test that fails if demographic disparity > 0.1.

In [13]:
def fairness_check(df, sensitive_attr, threshold=0.1):
    rates = df.groupby(sensitive_attr)["y_pred"].mean()
    disparity = rates.max() - rates.min()
    print(f"{sensitive_attr} Disparity: {round(disparity, 3)}")
    return disparity <= threshold

print("\nFairness Checks:")
gender_fair = fairness_check(df_test, "Gender")
ethnic_fair = fairness_check(df_test, "Ethnicity")

if not (gender_fair and ethnic_fair):
    print("❌ Fairness check FAILED — Mitigation required.")
else:
    print("✅ Fairness check PASSED.")


Fairness Checks:
Gender Disparity: 0.465
Ethnicity Disparity: 0.261
❌ Fairness check FAILED — Mitigation required.


#Task 3 — Feature Role & Ethics Review


    Generate feature importance rankings.
    Flag socio‑demographic features (Gender, Ethnicity) if they rank high.
    Suggest how to de‑bias them (e.g., removal, reweighting, representation balancing).


In [18]:
importances = pd.Series(model.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).head(10)

#importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
#print("\nTop Features (Importance):")
#print(importances.head(10))

,0
Skill_Score,0.298223
Age,0.272882
Years_Experience,0.233171
Gender_Male,0.039751
Ethnicity_Group_B,0.032707
Education_Masters,0.032055
Education_High School,0.028008
Ethnicity_Group_C,0.027460
Education_PhD,0.021553
Gender_Other,0.014190


#Task 4 — Ethics-Aware Agile User Stories

Document two user stories for your model development that explicitly incorporate ethics (e.g., "As a recruiter, I want the model to avoid bias based on gender").

Example user stories:

    US1: As a recruiter, I want the AI model to evaluate candidates without bias based on gender so that all applicants are treated fairly.
    US2: As a compliance officer, I want automated fairness checks in the model training pipeline so that biased models are not deployed.


#Part B — Operationalizing Ethics in Deployment

#Task 5 — Logging & Accountability

Create a prediction function that logs:

    Timestamp
    Input features
    Model output
    Model version


In [19]:
import datetime
prediction_log = []
MODEL_VERSION = "1.0.0"

def predict_and_log(features):
    pred = model.predict(features)[0]
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "model_version": MODEL_VERSION,
        "input": features.to_dict(orient='records')[0],
        "prediction": int(pred)
    }
    prediction_log.append(log_entry)
    return pred

# Simulate one prediction
sample_applicant = X_test.iloc[[0]]
predict_and_log(sample_applicant)

# Inspect logs
prediction_log[:3]

[{'timestamp': '2025-08-13T10:57:19.613112',
  'model_version': '1.0.0',
  'input': {'Age': 44,
   'Years_Experience': 1,
   'Skill_Score': 44,
   'Gender_Male': False,
   'Gender_Other': False,
   'Ethnicity_Group_B': False,
   'Ethnicity_Group_C': False,
   'Education_High School': True,
   'Education_Masters': False,
   'Education_PhD': False},
  'prediction': 0}]

#Task 6 — Real-Time Monitoring

Simulate post‑deployment monitoring:

    Track fairness disparity by Gender & Ethnicity.
    Trigger alerts if bounds are exceeded.


In [20]:
# Simulated post-deployment monitoring
dp_gender = df_test.groupby("Gender")["y_pred"].mean().max() - df_test.groupby("Gender")["y_pred"].mean().min()
dp_ethnicity = df_test.groupby("Ethnicity")["y_pred"].mean().max() - df_test.groupby("Ethnicity")["y_pred"].mean().min()

print(f"Gender Disparity: {dp_gender:.3f}")
print(f"Ethnicity Disparity: {dp_ethnicity:.3f}")

if dp_gender > 0.1 or dp_ethnicity > 0.1:
    print("⚠️ ALERT: Fairness threshold exceeded — Investigation required")
else:
    print("✅ Fairness levels within acceptable range")

Gender Disparity: 0.465
Ethnicity Disparity: 0.261
⚠️ ALERT: Fairness threshold exceeded — Investigation required


#Task 7 — Privacy Controls

In [21]:
import hashlib

data["Applicant_ID"] = data["Applicant_ID"].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
data.head()

,Applicant_ID,Age,Gender,Ethnicity,Education,Years_Experience,Skill_Score,Shortlisted
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,22,Male,Group_B,Bachelors,12,87,0
1,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,48,Other,Group_B,Bachelors,12,89,1
2,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,54,Male,Group_B,Bachelors,19,45,0
3,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,58,Male,Group_C,Masters,8,41,0
4,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,37,Male,Group_C,Bachelors,16,48,0


Apply pseudo‑ID hashing for Applicant_ID before storing predictions/logs.

#Task 8 — Inclusivity Feedback Loop

Simulate adding feedback cases from underrepresented groups to improve future retraining.

In [22]:
# Simulating underrepresented feedback cases
feedback = pd.DataFrame({
    "Age": [29, 32],
    "Gender": ["Other", "Other"],
    "Ethnicity": ["Group_C", "Group_C"],
    "Education": ["Bachelors", "Masters"],
    "Years_Experience": [5, 7],
    "Skill_Score": [82, 76],
    "Shortlisted": [1, 1]
})

# Append for future retraining
data_updated = pd.concat([data, feedback], ignore_index=True)
print("Updated dataset size:", data_updated.shape)

Updated dataset size: (1202, 8)


#Task 9 — Ethical Incident Simulation


    Create a mock scenario where a bias complaint is received.
    Write a structured incident response plan:
        Detection
        Investigation
        Communication
        Remediation


Bias Complaint Example Plan

    Detection — Monitoring dashboard flags Ethnicity bias > 0.15.
    Investigation — Review model logs & fairness metrics, audit training data composition.
    Communication — Report findings to compliance team & stakeholders.
    Remediation — Retrain with balanced data + remove sensitive feature weights.
    Prevention — Add stricter CI/CD bias thresholds.

